In [1]:
import pandas as pd
import keras
from keras.engine import Model
from keras.layers import Flatten, Dense, Input
from keras_vggface.vggface import VGGFace
import numpy as np
from keras.preprocessing import image
from keras_vggface import utils
import glob, re

Using TensorFlow backend.


In [2]:
model = VGGFace(model = "senet50", include_top=False, input_shape=(300, 300, 3), pooling='avg') # pooling: None, avg or max

In [5]:
#train_label_EI = pd.read_csv("train_E_I.csv", usecols=[1], delimiter=',').to_numpy().flatten()
#test_label_EI = pd.read_csv("test_E_I.csv", usecols=[1], delimiter=',').to_numpy().flatten()

train_label_MBTI = pd.read_csv("csv\\train_MBTI.csv", usecols=[1], delimiter=',').to_numpy().flatten()
test_label_MBTI = pd.read_csv("csv\\test_MBTI.csv", usecols=[1], delimiter=',').to_numpy().flatten()

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_label_MBTI = le.fit_transform(train_label_MBTI)
test_label_MBTI = le.transform(test_label_MBTI)

In [6]:
def preprocess_image(path):
    img = image.load_img(path, target_size=(300, 300))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = utils.preprocess_input(x, version=2)
    return x

In [7]:
#df_train_EI = pd.read_csv("train_E_I.csv", usecols=[0], delimiter=',').values.flatten()
#df_test_EI = pd.read_csv("test_E_I.csv", usecols=[0], delimiter=',').values.flatten()
#preprocessed_train_images_EI = np.vstack([preprocess_image(df_train_EI[i]) for i in range(df_train_EI.shape[0])])
#preprocessed_test_images_EI = np.vstack([preprocess_image(df_test_EI[i]) for i in range(df_test_EI.shape[0])])

df_train_MBTI = pd.read_csv("csv\\train_MBTI.csv", usecols=[0], delimiter=',').values.flatten()
df_test_MBTI = pd.read_csv("csv\\test_MBTI.csv", usecols=[0], delimiter=',').values.flatten()
preprocessed_train_images_MBTI = np.vstack([preprocess_image(df_train_MBTI[i]) for i in range(df_train_MBTI.shape[0])])
preprocessed_test_images_MBTI = np.vstack([preprocess_image(df_test_MBTI[i]) for i in range(df_test_MBTI.shape[0])])

In [8]:
#train_features_EI = model.predict(preprocessed_train_images_EI)
#train_features_EI = train_features_EI.reshape(train_label_EI.shape[0], -1)

#test_features_EI = model.predict(preprocessed_test_images_EI)
#test_features_EI = test_features_EI.reshape(test_label_EI.shape[0], -1)

train_data_MBTI = model.predict(preprocessed_train_images_MBTI)
train_data_MBTI = train_data_MBTI.reshape(train_label_MBTI.shape[0], -1)

test_data_MBTI = model.predict(preprocessed_test_images_MBTI)
test_data_MBTI = test_data_MBTI.reshape(test_label_MBTI.shape[0], -1)

In [9]:
#np.save("vgg\\train_EI.npy", vgg_train_features_EI)
#np.save("vgg\\test_EI.npy", vgg_test_features_EI)
#np.save("vgg\\label_train_EI.npy", train_label_EI)
#np.save("vgg\\label_test_EI.npy", test_label_EI)

np.save("vgg\\train_data_MBTI.npy", train_data_MBTI)
np.save("vgg\\test_data_MBTI.npy", test_data_MBTI)
np.save("vgg\\train_label_MBTI.npy", train_label_MBTI)
np.save("vgg\\test_label_MBTI.npy", test_label_MBTI)